In [3]:
import pm4py

log = pm4py.read_xes("logs/sepsis.xes")

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

In [4]:
from convert_log import convert_xes_n3

convert_xes_n3(log, "logs/sepsis.n3")

total number of traces: 1050
conversion time (ms): 2318.499
save time (ms): 2032.467
total time (ms): 4351.146
